In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

from magni_dash.data_preprocessing.spatio_temporal_features import SpatioTemporalFeatures
from magni_dash.utils.common import GroupsInfo, get_mapping_cols_tobii, get_mapping_cols, get_mapping_cols_centroids
from magni_dash.config.constants import TRAJECTORY_SAMPLES_PATH

In [2]:
files = os.listdir(os.path.join(TRAJECTORY_SAMPLES_PATH, "Scenario3"))
files_target = list(filter(lambda x: x.endswith("merged.csv"), files))
files_target

['Qualisys_180522_SC3A_R2_PP_6D_merged.csv']

In [3]:
FILE_IDX = 0

In [4]:
files_target[FILE_IDX]

'Qualisys_180522_SC3A_R2_PP_6D_merged.csv'

In [5]:
sync_df = pd.read_csv(os.path.join(TRAJECTORY_SAMPLES_PATH, "Scenario3", files_target[FILE_IDX]))

In [6]:
sync_df[sync_df.columns[sync_df.columns.str.contains("Helmet_10 Centroid")]].isna().sum() / sync_df.shape[0]

Helmet_10 Centroid_X    0.488528
Helmet_10 Centroid_Y    0.488528
Helmet_10 Centroid_Z    0.488528
dtype: float64

In [7]:
for m in range(1,5):
    print("X", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} X")]].isna().sum() / sync_df.shape[0])
    print("Y", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} Y")]].isna().sum() / sync_df.shape[0])
    print("Z", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} Z")]].isna().sum() / sync_df.shape[0])

X Helmet_10 - 1 X    0.082167
dtype: float64
Y Helmet_10 - 1 Y    0.082167
dtype: float64
Z Helmet_10 - 1 Z    0.082167
dtype: float64
X Helmet_10 - 2 X    0.264847
dtype: float64
Y Helmet_10 - 2 Y    0.264847
dtype: float64
Z Helmet_10 - 2 Z    0.264847
dtype: float64
X Helmet_10 - 3 X    0.238768
dtype: float64
Y Helmet_10 - 3 Y    0.238768
dtype: float64
Z Helmet_10 - 3 Z    0.238768
dtype: float64
X Helmet_10 - 4 X    0.247377
dtype: float64
Y Helmet_10 - 4 Y    0.247377
dtype: float64
Z Helmet_10 - 4 Z    0.247377
dtype: float64


In [8]:
sync_df[sync_df.columns[sync_df.columns.str.contains("G3D")]].isna().sum() / sync_df.shape[0]

Helmet_10 TB2_G3D_X    0.754796
Helmet_10 TB2_G3D_Y    0.754796
Helmet_10 TB2_G3D_Z    0.754796
Helmet_5 TB3_G3D_X     0.761232
Helmet_5 TB3_G3D_Y     0.761232
Helmet_5 TB3_G3D_Z     0.761232
dtype: float64

In [9]:
def get_best_markers(input_df: pd.DataFrame):
    """Get markers with lowest amount of NaN values"""
    x_coordinate = input_df[input_df.columns[input_df.columns.str.endswith("X")]]
    x_cols = x_coordinate.columns

    instances = set(x_coordinate.columns.str.split(" - ").str[0])
    instances = list(filter(lambda x: len(x.split(" ")) == 1, instances))
    nan_counter_by_marker = {}
    for instance_id in instances:
        nan_counter_by_marker[instance_id] = {}
        markers = (
            x_coordinate[x_cols[x_cols.str.startswith(f"{instance_id} -")]]
            .columns.str.split(regex=r" (/d) ")
            .str[2]
        )
        for marker_id in markers:
            n_nans = (
                x_coordinate[f"{instance_id} - {marker_id} X"]
                .isna()
                .sum()
            )
            nan_counter_by_marker[instance_id][marker_id] = n_nans
    print(nan_counter_by_marker)
    return nan_counter_by_marker

In [10]:
best_markers = get_best_markers(sync_df)

{'Helmet_5': {'1': 6014, '2': 731, '3': 7070, '4': 8713, '5': 6359}, 'Helmet_1': {'1': 3175, '2': 1078, '3': 1782}, 'Helmet_4': {'1': 3289, '2': 9533, '3': 8395, '4': 10254, '5': 11596}, 'Helmet_7': {'1': 4824, '2': 10979, '3': 7166, '4': 14268, '5': 8054}, 'Helmet_6': {'1': 695, '2': 11275, '3': 7188, '4': 7870, '5': 8118}, 'Helmet_10': {'1': 1966, '2': 6337, '3': 5713, '4': 5919}, 'DARKO': {'1': 375, '2': 364, '3': 180, '4': 233, '5': 455, '6': 648, '7': 679}, 'LO1': {'1': 1639, '2': 1595, '3': 1062, '4': 922, '5': 1803, '6': 1536, '7': 1051, '8': 1930}, 'Helmet_2': {'1': 2599, '2': 2236, '3': 2782, '4': 3469}}


In [11]:
def preprocess_df(raw_df: pd.DataFrame) -> pd.DataFrame:
    """interpolation and divide by 1000 to get measurements in meters"""
    preprocessed_df = raw_df.copy()
    trajectories_condition = preprocessed_df.columns[
            (preprocessed_df.columns.str.endswith(" X"))
            | ((preprocessed_df.columns.str.endswith(" Y")))
            | ((preprocessed_df.columns.str.endswith(" Z")))
        ]
    preprocessed_df[trajectories_condition] = preprocessed_df[trajectories_condition].interpolate()
    preprocessed_df[trajectories_condition] /= 1000
    return preprocessed_df

In [12]:
sync_df[sync_df.columns[sync_df.columns.str.endswith(" X")]].columns

Index(['Helmet_10 - 1 X', 'Helmet_10 - 2 X', 'Helmet_10 - 3 X',
       'Helmet_10 - 4 X', 'Helmet_5 - 1 X', 'Helmet_5 - 2 X', 'Helmet_5 - 3 X',
       'Helmet_5 - 4 X', 'Helmet_5 - 5 X', 'Helmet_1 - 1 X', 'Helmet_1 - 2 X',
       'Helmet_1 - 3 X', 'Helmet_2 - 1 X', 'Helmet_2 - 2 X', 'Helmet_2 - 3 X',
       'Helmet_2 - 4 X', 'Helmet_4 - 1 X', 'Helmet_4 - 2 X', 'Helmet_4 - 3 X',
       'Helmet_4 - 4 X', 'Helmet_4 - 5 X', 'Helmet_6 - 1 X', 'Helmet_6 - 2 X',
       'Helmet_6 - 3 X', 'Helmet_6 - 4 X', 'Helmet_6 - 5 X', 'Helmet_7 - 1 X',
       'Helmet_7 - 2 X', 'Helmet_7 - 3 X', 'Helmet_7 - 4 X', 'Helmet_7 - 5 X',
       'DARKO - 1 X', 'DARKO - 2 X', 'DARKO - 3 X', 'DARKO - 4 X',
       'DARKO - 5 X', 'DARKO - 6 X', 'DARKO - 7 X', 'LO1 - 1 X', 'LO1 - 2 X',
       'LO1 - 3 X', 'LO1 - 4 X', 'LO1 - 5 X', 'LO1 - 6 X', 'LO1 - 7 X',
       'LO1 - 8 X'],
      dtype='object')

In [13]:
preprocessed_trajectories = preprocess_df(sync_df)

In [14]:
preprocessed_trajectories[
    preprocessed_trajectories.columns[
        preprocessed_trajectories.columns.str.contains("G3D")
    ]
].isna().sum() / preprocessed_trajectories.shape[0]

Helmet_10 TB2_G3D_X    0.754796
Helmet_10 TB2_G3D_Y    0.754796
Helmet_10 TB2_G3D_Z    0.754796
Helmet_5 TB3_G3D_X     0.761232
Helmet_5 TB3_G3D_Y     0.761232
Helmet_5 TB3_G3D_Z     0.761232
dtype: float64

In [15]:
moving_agents = preprocessed_trajectories.columns[
    (preprocessed_trajectories.columns.str.startswith("Helmet"))
    | (preprocessed_trajectories.columns.str.startswith("LO1"))
].tolist()
moving_agents_labels = set(map(lambda x: x.split(" - ")[0], moving_agents))
moving_agents_labels = filter(lambda x: len(x.split(" ")) == 1, moving_agents_labels)

In [16]:
def extract_features(
    out_df: pd.DataFrame,
    magents_labels,
    darko_label,
) -> pd.DataFrame:
    """Extract features to be used in the profiles section such as speed

    Parameters
    ----------
    input_df
        raw pandas DataFrame
    magents_labels
        moving agents labels
    darko_label
        darko robot label

    Returns
    -------
        pandas DataFrame with the respective features computed
    """
    magents_labels = (
        [magents_labels] if isinstance(magents_labels, str) else magents_labels
    )
    elements_labels = magents_labels + [darko_label] if darko_label else magents_labels

    out_df = SpatioTemporalFeatures.get_speed(
        out_df,
        time_col_name="Time",
        element_name=elements_labels,
    )

    out_df = out_df.reset_index()
    return out_df

In [17]:
moving_agents_cols = preprocessed_trajectories.columns[(preprocessed_trajectories.columns.str.endswith(" X"))
            | ((preprocessed_trajectories.columns.str.endswith(" Y")))
            | ((preprocessed_trajectories.columns.str.endswith(" Z")))]

In [18]:
moving_agents_cols

Index(['Helmet_10 - 1 X', 'Helmet_10 - 1 Y', 'Helmet_10 - 1 Z',
       'Helmet_10 - 2 X', 'Helmet_10 - 2 Y', 'Helmet_10 - 2 Z',
       'Helmet_10 - 3 X', 'Helmet_10 - 3 Y', 'Helmet_10 - 3 Z',
       'Helmet_10 - 4 X',
       ...
       'LO1 - 5 Z', 'LO1 - 6 X', 'LO1 - 6 Y', 'LO1 - 6 Z', 'LO1 - 7 X',
       'LO1 - 7 Y', 'LO1 - 7 Z', 'LO1 - 8 X', 'LO1 - 8 Y', 'LO1 - 8 Z'],
      dtype='object', length=138)

In [19]:
features_df = extract_features(
        preprocessed_trajectories[["Time"] + moving_agents_cols.tolist()].copy(),
        magents_labels=list(moving_agents_labels),
        darko_label="DARKO",
    )

2023-05-13 14:53:08.794 INFO    magni_dash.data_preprocessing.spatio_temporal_features: {'Helmet_5 - 3 X_delta', 'LO1 - 5 X_delta', 'Helmet_6 - 5 Z_delta', 'LO1 - 4 Z_delta', 'Helmet_4 - 5 Z_delta', 'Helmet_4 - 5 Y_delta', 'Helmet_5 - 5 X_delta', 'LO1 - 7 Z_delta', 'Helmet_7 - 1 Z_delta', 'DARKO - 2 Z_delta', 'LO1 - 1 Y_delta', 'Helmet_1 - 1 Z_delta', 'DARKO - 6 Z_delta', 'Helmet_10 - 2 Y_delta', 'Helmet_10 - 2 X_delta', 'Time_delta', 'LO1 - 6 Z_delta', 'Helmet_1 - 2 Y_delta', 'Helmet_6 - 4 X_delta', 'Helmet_7 - 1 Y_delta', 'Helmet_6 - 3 Y_delta', 'LO1 - 1 Z_delta', 'Helmet_7 - 2 Y_delta', 'Helmet_6 - 2 Z_delta', 'Helmet_4 - 3 Z_delta', 'Helmet_7 - 4 Z_delta', 'Helmet_10 - 1 X_delta', 'Helmet_7 - 3 Z_delta', 'LO1 - 2 Y_delta', 'Helmet_10 - 1 Y_delta', 'Helmet_1 - 3 Y_delta', 'LO1 - 4 Y_delta', 'DARKO - 2 Y_delta', 'Helmet_5 - 4 Z_delta', 'LO1 - 8 Y_delta', 'DARKO - 7 Y_delta', 'Helmet_4 - 3 X_delta', 'Helmet_10 - 4 Z_delta', 'Helmet_5 - 1 X_delta', 'Helmet_6 - 3 X_delta', 'Helmet_6 - 5

In [20]:
features_df.shape

(23927, 240)

In [21]:
features_cat = preprocessed_trajectories.join(features_df)

In [22]:
features_filtered = features_cat[
    ["Frame"] + 
    features_cat.columns[
        (features_cat.columns.str.endswith("X"))
        | (features_cat.columns.str.endswith("Y"))
        | (features_cat.columns.str.endswith("Z"))
        | (features_cat.columns.str.endswith("speed (m/s)"))
    ].tolist()
]

In [23]:
features_filtered

,Frame,Helmet_10 - 1 X,Helmet_10 - 1 Y,Helmet_10 - 1 Z,Helmet_10 - 2 X,Helmet_10 - 2 Y,Helmet_10 - 2 Z,Helmet_10 - 3 X,Helmet_10 - 3 Y,Helmet_10 - 3 Z,...,Helmet_1 - 1 speed (m/s),Helmet_1 - 2 speed (m/s),Helmet_1 - 3 speed (m/s),DARKO - 1 speed (m/s),DARKO - 2 speed (m/s),DARKO - 3 speed (m/s),DARKO - 4 speed (m/s),DARKO - 5 speed (m/s),DARKO - 6 speed (m/s),DARKO - 7 speed (m/s)
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.076438,0.000000,0.141272,0.149300,0.660975,0.149439,0.228801,0.151343,0.144788
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.116661,0.000000,0.140485,0.138816,0.292588,0.274291,0.168218,0.150351,0.149162
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.136647,0.000000,0.170130,0.139797,0.434471,0.228035,0.143600,0.936443,0.143234
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.152709,0.000000,0.131247,0.262515,0.132606,0.140941,0.142135,0.143229,0.143480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23922,23923,8.116425,0.857656,1.964417,7.856372,0.887048,1.954488,6.432516,0.710742,1.913089,...,0.0,0.678417,1.084825,0.066267,0.066580,0.063015,0.061349,0.064413,0.067585,0.066238
23923,23924,8.125735,0.859631,1.964619,7.856372,0.887048,1.954488,6.432516,0.710742,1.913089,...,0.0,0.709114,0.917435,0.072272,0.076900,0.063667,0.068374,0.063673,0.078714,0.076701
23924,23925,8.136522,0.862119,1.964291,7.856372,0.887048,1.954488,6.432516,0.710742,1.913089,...,0.0,0.675818,0.688343,0.083697,0.075700,0.081948,0.074617,0.144948,0.073436,0.072629
23925,23926,8.149063,0.865221,1.963243,7.856372,0.887048,1.954488,6.432516,0.710742,1.913089,...,0.0,0.650360,0.000000,0.098399,0.092731,0.082215,0.088135,0.085600,0.091817,0.080775


# Visualization

In [24]:
import plotly.express as px
import plotly.graph_objects as go

In [25]:
features_filtered.columns[features_filtered.columns.str.contains("G3D")]

Index(['Helmet_10 TB2_G3D_X', 'Helmet_10 TB2_G3D_Y', 'Helmet_10 TB2_G3D_Z',
       'Helmet_5 TB3_G3D_X', 'Helmet_5 TB3_G3D_Y', 'Helmet_5 TB3_G3D_Z'],
      dtype='object')

In [26]:
HELMET_TO_VISUALIZE = "Helmet_10"

In [27]:
gaze_data = features_filtered[
    [
        f"{HELMET_TO_VISUALIZE} TB2_G3D_X",
        f"{HELMET_TO_VISUALIZE} TB2_G3D_Y",
        f"{HELMET_TO_VISUALIZE} TB2_G3D_Z",
    ]
].dropna()
gaze_data.shape

(5867, 3)

In [28]:
centroids_data = features_filtered[features_filtered.columns[features_filtered.columns.str.contains(f"{HELMET_TO_VISUALIZE} Centroid")]].dropna()
centroids_data.shape

(12238, 3)

In [29]:
centroids_data

,Helmet_10 Centroid_X,Helmet_10 Centroid_Y,Helmet_10 Centroid_Z
891,-8295.28998,-993.88804,1882.33080
892,-8294.53301,-994.77403,1882.37496
893,-8291.72100,-995.63761,1882.68680
894,-8289.29061,-996.29318,1883.40001
895,-8286.48089,-997.36220,1883.73236
...,...,...,...
23444,5365.74768,1190.43576,1904.95191
23445,5371.73859,1184.23697,1905.76156
23446,5377.76340,1178.04349,1909.05072
23447,5383.43407,1171.62597,1909.93495


In [30]:
centroids_filtered = centroids_data.loc[gaze_data.index]
centroids_filtered /= 1000
centroids_filtered.shape

(5867, 3)

In [52]:
lo_info = GroupsInfo(
    element_id="LO1", markers_pattern_re=r"LO1 - (\d).*", label_sep=" - "
)
darko_info = GroupsInfo(
    element_id="DARKO", markers_pattern_re=r"DARKO - (\d).*", label_sep=" - "
)
helmets_info = GroupsInfo(
    element_id="Helmet", markers_pattern_re=r"Helmet_(\d+ - \d).*", label_sep="_"
)
tobii_info = GroupsInfo(
    element_id="Helmet", markers_pattern_re=r"Helmet_(\d+ TB\d)_G3D.*", label_sep="_"
)
centroids_info = GroupsInfo(
        element_id="Helmet",
        markers_pattern_re=r"Helmet_(\d+) Centroid_.*",
        label_sep="_",
    )


In [58]:
def transform_df2plotly(
    input_df: pd.DataFrame, groups_info
) -> pd.DataFrame:
    """Transform a dataframe into the plotly best suited format
    |   Frame    |   X (m)  |   Y (m)  |   eid   |   mid   |

    being `eid` the element identifier (e.g. Helmet, DARKO, etc), and `mid` the marker identifier

    Parameters
    ----------
    input_df
        input pandas DataFrame
    element_id
        see eid explanation above
    markers_pattern_re
        regex to groupby element id and markers
    sep
        separation used in col name form element id and marker id

    Returns
    -------
        Transformed pandas DataFrame
    """
    groups_info = [groups_info] if isinstance(groups_info, GroupsInfo) else groups_info
    groups = []
    for group_info in groups_info:
        element_id = group_info.element_id
        elements_grouped = input_df.groupby(
            input_df.columns.str.extract(group_info.markers_pattern_re, expand=False),
            axis=1,
        )
        for group_name, group in elements_grouped:
            if element_id == "Helmet" and len(group_name.split("-")) == 1:
                tobii = len(group_name.split(" ")) == 2
                 # eyt or centroids
                _mapping_cols = get_mapping_cols_tobii(
                    element_id,
                    group_name,
                    group_info.label_sep,
                ) if tobii else get_mapping_cols_centroids(element_id,
                    group_name,
                    group_info.label_sep)
                group = group.rename(_mapping_cols, axis=1)
                eid = element_id + "_" + group_name.split(" ")[0]
            else:
                _mapping_cols = get_mapping_cols(
                    element_id, group_name, group_info.label_sep
                )
                group = group.rename(_mapping_cols, axis=1)
                eid = (
                    element_id + "_" + group_name.split(" - ")[0]
                    if element_id == "Helmet"
                    else element_id
                )
                mid = group_name.split(" - ")[1] if element_id == "Helmet" else group_name
                group["mid"] = mid
            group["eid"] = eid
            groups.append(group)
    out_df = pd.concat(groups, axis=0)
    return out_df

In [64]:
transformed = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[tobii_info],
    )

In [65]:
transformed.eid

0        Helmet_10
1        Helmet_10
2        Helmet_10
3        Helmet_10
4        Helmet_10
           ...    
23922     Helmet_5
23923     Helmet_5
23924     Helmet_5
23925     Helmet_5
23926     Helmet_5
Name: eid, Length: 47854, dtype: object

In [35]:
transformed_trajectories = transform_df2plotly(
        input_df=features_filtered[features_filtered.columns[features_filtered.columns.str.contains("")]].copy(),
        groups_info=[helmets_info],
    )
transformed_trajectories = transformed_trajectories.sort_index()

In [36]:
transformed_trajectories

,X (m),Y (m),Z (m),speed (m/s),mid,eid
0,NaN,NaN,NaN,NaN,1,Helmet_1
0,NaN,NaN,NaN,NaN,2,Helmet_10
0,NaN,NaN,NaN,NaN,2,Helmet_6
0,0.359082,3.059350,1.799919,NaN,3,Helmet_2
0,NaN,NaN,NaN,NaN,4,Helmet_10
...,...,...,...,...,...,...
23926,-8.317673,1.921456,1.497061,0.000000,5,Helmet_5
23926,8.163636,0.869036,1.961281,1.519131,1,Helmet_10
23926,8.933929,-0.531036,1.544316,0.787617,1,Helmet_2
23926,8.504732,-0.501440,1.683069,0.000000,5,Helmet_6


In [39]:
transformed.sort_index()

,TB_G3D X (m),TB_G3D Y (m),TB_G3D Z (m),eid,Centroid X (m),Centroid Y (m),Centroid Z (m)
0,NaN,NaN,NaN,Helmet_10,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_5,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_7,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_5,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_1,NaN,NaN,NaN
...,...,...,...,...,...,...,...
23926,NaN,NaN,NaN,Helmet_10,NaN,NaN,NaN
23926,NaN,NaN,NaN,Helmet_1,NaN,NaN,NaN
23926,NaN,NaN,NaN,Helmet_4,NaN,NaN,NaN
23926,NaN,NaN,NaN,Helmet_6,NaN,NaN,NaN


In [42]:
transformed["Centroid X (m)"].dropna()

24      -7266.26826
25      -7269.02931
26      -7269.70511
27      -7271.02460
28      -7273.19973
            ...    
23532    4807.03929
23533    4817.82989
23534    4827.75654
23536    4849.18181
23537    4856.99515
Name: Centroid X (m), Length: 87269, dtype: float64

In [68]:
def filter_best_markers(elements_cat_df: pd.DataFrame, nan_counter_by_marker):
    elements_filtered_by_best_marker = []
    for instance_id, nans_counter in nan_counter_by_marker.items():
        best_marker_id = min(
            nans_counter,
            key=nans_counter.get,
        )
        elements_filtered_by_best_marker.append(
            elements_cat_df[
                (elements_cat_df.eid == instance_id)
                & (elements_cat_df.mid == best_marker_id)
            ]
        )
    out_df = pd.concat(elements_filtered_by_best_marker, axis=0)
    out_df = out_df.sort_index().reset_index()
    return out_df

In [69]:
best_makers_df = filter_best_markers(
        elements_cat_df=transformed_trajectories.copy(), nan_counter_by_marker=best_markers
    )

In [70]:
best_makers_df

,index,X (m),Y (m),Z (m),speed (m/s),eid,mid
0,0,NaN,NaN,NaN,NaN,Helmet_4,1
1,0,-8.414543,1.655896,1.353073,NaN,Helmet_5,2
2,0,0.236878,2.938533,1.829624,NaN,Helmet_2,2
3,0,NaN,NaN,NaN,NaN,Helmet_7,1
4,0,-7.037703,-2.437462,1.812973,NaN,Helmet_1,2
...,...,...,...,...,...,...,...
167484,23926,8.561741,0.561951,1.885170,0.603637,Helmet_1,2
167485,23926,7.683523,0.036912,1.863660,0.876024,Helmet_4,1
167486,23926,8.998510,-0.572453,1.623232,1.073293,Helmet_2,2
167487,23926,-8.583218,2.080728,1.261840,0.960324,Helmet_5,2


In [71]:
ORIGIN = (0,0,0)
X = (1, 0, 0)
Y = (0, 1, 0)
Z = (0, 0, 1)
COLOR1 = 'red'
COLOR2 = 'green'
COLOR3 = 'blue'
COLOR4 = 'orange'

<TB2_G3D_X, TB2_G3D_Y, TB2_G3D_Z> is the transformation from the helmet frame to a convergence point
of the eye gaze.

In [73]:
# fig = px.scatter_3d(
#     transformed[:40000],
#     x="TB_G3D X (m)",
#     y="TB_G3D Y (m)",
#     z="TB_G3D Z (m)",
#     color="eid"
# )
fig = go.Figure([])
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], X[0], None],
        y=[ORIGIN[1], X[1], None],
        z=[ORIGIN[2], X[2], None],
        mode="lines",
        line=dict(color="red", width=5),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], Y[0], None],
        y=[ORIGIN[1], Y[1], None],
        z=[ORIGIN[2], Y[2], None],
        mode="lines",
        line=dict(color="green", width=5),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], Z[0], None],
        y=[ORIGIN[1], Z[1], None],
        z=[ORIGIN[2], Z[2], None],
        mode="lines",
        line=dict(color="blue", width=5),
        showlegend=False,
    )
)

fig.add_cone(
    x=[1],
    y=[0],
    z=[0],
    u=[1],
    v=[0],
    w=[0],
    colorscale=[[0, COLOR1], [1, COLOR1]],
    showscale=False,
)

fig.add_cone(
    x=[0],
    y=[1],
    z=[0],
    u=[0],
    v=[1],
    w=[0],
    colorscale=[[0, COLOR2], [1, COLOR2]],
    showscale=False,
)

fig.add_cone(
    x=[0],
    y=[0],
    z=[Z[2]],
    u=[0],
    v=[0],
    w=[Z[2]],
    colorscale=[[0, COLOR3], [1, COLOR3]],
    showscale=False,
)